<a href="https://colab.research.google.com/github/eduion/AIOT/blob/main/HW5/HW5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [12]:
# 定義分割比例
train_split, valid_split, test_split = ['train[:90%]', 'train[90%:95%]', 'train[95%:]']

# 定義 parse_fn
def parse_fn(dataset):
    x = tf.cast(dataset['features'], tf.float32)  # 特徵為 'features'
    y = tf.one_hot(dataset['label'], 3)  # 標籤為 'label'，共有 3 類
    return x, y

# 加載數據集
train_data, info = tfds.load("iris", split=train_split, with_info=True)
valid_data = tfds.load("iris", split=valid_split)
test_data = tfds.load("iris", split=test_split)

In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE  # 自動調整模式
batch_size = 64  # 批次大小
train_num = int(info.splits['train'].num_examples / 10) * 9  # 訓練資料數量
# 處理訓練資料
train_data = train_data.shuffle(train_num)\
                      .map(parse_fn, num_parallel_calls=AUTOTUNE)\
                       .batch(batch_size)\
                       .prefetch(buffer_size=AUTOTUNE)

# 處理驗證資料
valid_data = valid_data.map(parse_fn, num_parallel_calls=AUTOTUNE)\
                       .batch(batch_size) \
                       .prefetch(buffer_size=AUTOTUNE)

# 處理測試資料
test_data = test_data.map(parse_fn, num_parallel_calls=AUTOTUNE) \
                     .batch(batch_size) \
                     .prefetch(buffer_size=AUTOTUNE)

# 確認資料流程正常
for features, labels in train_data.take(1):
    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)


Features shape: (64, 4)
Labels shape: (64, 3)


In [15]:
# 定義輸入層
inputs = keras.Input(shape=(4,), name='iris_input')

# 添加隱藏層
x = layers.Dense(128, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)  # 避免過擬合
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# 定義輸出層
outputs = layers.Dense(3, activation='softmax', name='iris_output')(x)

# 建立模型
model_iris = keras.Model(inputs, outputs, name='iris_model')

# 查看模型摘要
model_iris.summary()

Model: "iris_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ iris_input (InputLayer)              │ (None, 4)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ iris_output (Dense)                  │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,075 (43.26 KB)

 Trainable params: 11,075 (43.26 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# 創建儲存權重目錄
model_dir = 'iris-logs/models'
os.makedirs(model_dir, exist_ok=True)

# 建立 Callback function
log_dir = os.path.join('iris-logs', 'model-1')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(model_dir, 'Best-model-1.keras'),  # 修改副檔名為 .keras
    monitor='val_categorical_accuracy',
    save_best_only=True,
    mode='max'
)

In [25]:
# 設定優化器、損失函數和指標
model_iris.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()]
)

# 訓練模型
history_iris = model_iris.fit(
    train_data,
    epochs=100,
    validation_data=valid_data,
    callbacks=[model_cbk, model_mckp]
)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - categorical_accuracy: 0.9713 - loss: 0.1101 - val_categorical_accuracy: 0.8571 - val_loss: 0.2017
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - categorical_accuracy: 0.9372 - loss: 0.2095 - val_categorical_accuracy: 1.0000 - val_loss: 0.0842
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - categorical_accuracy: 0.9411 - loss: 0.1623 - val_categorical_accuracy: 0.8571 - val_loss: 0.3113
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - categorical_accuracy: 0.9200 - loss: 0.1574 - val_categorical_accuracy: 1.0000 - val_loss: 0.1357
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - categorical_accuracy: 0.9465 - loss: 0.1293 - val_categorical_accuracy: 1.0000 - val_loss: 0.0731
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - categorical_accuracy: 0.9659 - loss: 0.1567 - val_categorical_accuracy: 1.0000 - val_loss: 0.0776
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - categorical_accuracy: 0.9315 - loss: 0.1697 - va

In [26]:
loss, acc = model_iris.evaluate(test_data)
print('\nModel-3 Accuracy: {}%'.format(acc))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - categorical_accuracy: 1.0000 - loss: 0.0031

Model-3 Accuracy: 1.0%


In [27]:
loss = [loss]
acc = [acc]

dict = {"test_Loss": loss,
        "test_Accuracy": acc}

print(pd.DataFrame(dict))

   test_Loss  test_Accuracy
0   0.003136            1.0
